In [3]:
from langchain_community.llms import Ollama
import json
import random
import re
import os

# Combine several papers

In [5]:

# The original state of the art paper
paper_id = "2402.01383v1"


In [17]:
def get_json(file_name):
    # Step 1: Read the JSON file
    with open(file_name + '.json', 'r') as file:
        json_data = json.load(file)
    return json_data
def write_json(dict_data,file_name):
    with open('summaries/'+file_name+'.json', 'w') as file:
        json.dump(dict_data, file, indent=4)



#store the summaries
os.makedirs('summaries/'+paper_id,exist_ok=True)
def store_summary(content,filename):
    path = 'summaries/'+paper_id
    with open(path+'/'+filename, 'w') as file:
        file.write(content)

In [7]:
original = get_json('dataset/'+paper_id+'data')
topic = original['title']
topic

'LLM-based NLG Evaluation: Current Status and Challenges'

In [8]:
#Get full texts of related papers
data = get_json('dataset/'+paper_id+'full_texts')

In [9]:
# Get the llm mistral
llm = Ollama(model = "mistral",temperature=0)

In [10]:
def summarize_n(papers):
    l = [key+':  '+value for key,value in papers.items()]
    random.shuffle(l)
    context = ('/n/n').join(l)
    instruction = 'summarize the following papers:/n'+context+'/n summarize them.'
    a = llm.invoke(instruction)
    return a

In [11]:
longsummary = summarize_n(data)
longsummary
#type(random.shuffle([key+':  '+value for key,value in data.items()]))
#complete fail

' The first answer about defensive strategies in basketball is rated as having a score of 3 out of 5. The accuracy of the answer is assessed as 3, indicating that it correctly identifies the World Health Organization (WHO) as the primary organization responsible for global health policies and guidelines. However, the coherence, factuality, and comprehensiveness scores are lower due to some errors in the description of defensive strategies and a lack of detail about the process by which WHO develops and implements these policies.\n\nThe second answer regarding attachment styles and romantic relationships is rated as having a score of 4 out of 5. The accuracy of the answer is assessed as 3, indicating that it correctly describes how attachment style influences romantic relationships. However, the coherence, factuality, and comprehensiveness scores are higher due to the clear and detailed explanation of each attachment style and its impact on relationships.\n\nThe third question asks abou

In [13]:
store_summary(longsummary,'fulltext_summary.txt')

In [14]:
#Is the problem that context is too big? 
l = [key+':  '+value for key,value in data.items()]
context = ('/n/n').join(l)
len(context) #3'924'350 characters. This is way too big as 
len(context.split(' ')) # 499'106 words but only 128'000 in theory are allowed. The llm was trained with 8'000 words in the prompt.
#yes

499106

### Try with only abstracts. Is it short enough?

In [18]:
#Get full texts of related papers
data = get_json('dataset/'+paper_id+'ref')

In [19]:
l = [value['title']+':  '+value['abstract'] for value in data.values()]
context = ('/n/n').join(l)
len(context.split(' ')) 
#7956 is short enough

7956

In [20]:
def write_summary(summaries):
    texts = ('\n').join([s['title']+':  '+s['abstract'] for s in summaries.values()])
    instruction = f'Write a summary combining the key findings of following texts: \n {texts}'
    a = llm.invoke(instruction)
    return a

In [21]:
simple_summary = write_summary(data)
simple_summary

' This text discusses several research papers related to evaluating and using large language models (LLMs) as judges for various tasks. The first paper explores the use of wider and deeper networks for fairer LLM evaluations, drawing inspiration from deep neural networks and academic paper reviewing. The second paper proposes using strong LLMs as judges for evaluating other LLMs on more open-ended questions and introduces two benchmarks: MT-bench and Chatbot Arena. The third paper proposes fine-tuning LLMs as scalable judges (JudgeLM) to evaluate LLMs efficiently and effectively in open-ended benchmarks.\n\nThe first paper, "Wider and Deeper LLM Networks are Fairer LLM Evaluators," discusses the challenges of evaluating the quality of responses generated by LLMs and proposes a method using an LLM itself to make evaluations through multiple independent evaluations. The authors explore whether deeper and wider networks can lead to fairer evaluations, inspired by the observation that diff

In [22]:
store_summary(simple_summary,'simple_summary.txt')

In [23]:
print(('\n').join([s['title'] for s in data.values()]))
#The first paper, "Wider and Deeper LLM Networks are Fairer LLM Evaluators,"... 
#=> it did only consider the last three papers

Concrete Problems in AI Safety
PaLM 2 Technical Report
Benchmarking Foundation Models with Language-Model-as-an-Examiner
ChatEval: Towards Better LLM-based Evaluators through Multi-Agent Debate
BooookScore: A systematic exploration of book-length summarization in the era of LLMs
Scaling Instruction-Finetuned Language Models
RAGAS: Automated Evaluation of Retrieval Augmented Generation
GPTScore: Evaluate as You Desire
Are Large Language Models Reliable Judges? A Study on the Factuality Evaluation Capabilities of LLMs
CoAScore: Chain-of-Aspects Prompting for NLG Evaluation
ALLURE: Auditing and Improving LLM-based Evaluation of Text using Iterative In-Context-Learning
SocREval: Large Language Models with the Socratic Method for Reference-Free Reasoning Evaluation
Exploring ChatGPT's Ability to Rank Content: A Preliminary Study on Consistency with Human Preferences
TIGERScore: Towards Building Explainable Metric for All Text Generation Tasks
CritiqueLLM: Towards an Informative Critique Gen

In [24]:
def write_summary_with_topic(summaries,topic):
    texts = ('\n').join([s['title']+':  '+s['abstract'] for s in summaries.values()])
    instruction = f'Use this {texts} to create an overview over the topic {topic}.'
    a = llm.invoke(instruction)
    return a

In [25]:
simple_overview = write_summary_with_topic(data,topic)
store_summary(simple_overview,'simple_overview.txt')
simple_overview

' Title: LLM-based NLG Evaluation: Current Status and Challenges - Wider and Deeper Networks for Fairer Evaluators, LLM-as-a-Judge, and JudgeLM\n\nThis topic covers recent research on evaluating Large Language Models (LLMs) in Natural Language Generation (NLG) tasks. The studies explore various approaches to improve the evaluation process, including wider and deeper networks for fairer evaluators, using LLMs as judges, and fine-tuning LLMs as scalable judges.\n\n1. Wider and Deeper LLM Networks are Fairer LLM Evaluators:\nThis paper proposes a novel approach to use the LLM itself to make evaluations and improve fairness by designing a network that resembles academic paper reviewing. The network consists of multiple layers, with each layer receiving representations from all neurons in the previous layer, integrating locally learned evaluation information for more comprehensive results. Experimental results demonstrate that a wider network (involving many reviewers) with 2 layers perform

In [26]:
#As well only considers the last 3 papers. Is it because they are the last three papers or because they are in llm's opinion the most relevant?

In [33]:
def write_summary_with_shuffle(summaries,topic):
    l = [s['title']+':  '+s['abstract'] for s in summaries.values()]
    random.shuffle(l)
    print('last 3 papers:',l[-3])
    print(l[-2])
    print(l[-1])
    texts = ('\n').join(l)
    instruction = f'Use this {texts} to create an overview over the topic {topic}.'
    a = llm.invoke(instruction)
    return a

In [34]:
simple_overview_s = write_summary_with_shuffle(data,topic)
store_summary(simple_overview_s,'simple_overview_shuffled.txt')
simple_overview_s

last 3 papers: Branch-Solve-Merge Improves Large Language Model Evaluation and Generation:  Large Language Models (LLMs) are frequently used for multi-faceted language
generation and evaluation tasks that involve satisfying intricate user
constraints or taking into account multiple aspects and criteria. However,
their performance can fall short, due to the model's lack of coherence and
inability to plan and decompose the problem. We propose Branch-Solve-Merge
(BSM), a Large Language Model program (Schlag et al., 2023) for tackling such
challenging natural language tasks. It consists of branch, solve, and merge
modules that are parameterized with specific prompts to the base LLM. These
three modules plan a decomposition of the task into multiple parallel
sub-tasks, independently solve them, and fuse the solutions to the sub-tasks.
We apply our method to the tasks of LLM response evaluation and constrained
text generation and evaluate its effectiveness with multiple LLMs, including
Vicun

" Title: Recent Advances in Large Language Model Evaluation: TIGERScore, EvalLM, Branch-Solve-Merge, and FLASK\n\nIn recent years, there has been a surge of interest in evaluating the performance of large language models (LLMs) in natural language generation (NLG) tasks. In this summary, we will discuss four recent studies that have made significant strides in developing new methods for evaluating LLMs: TIGERScore, EvalLM, Branch-Solve-Merge, and FLASK.\n\n1. TIGERScore: A Universal Explainable Metric for NLG Tasks\nTIGERScore is a reference-free metric that evaluates the rationality of explanations generated by LLMs. The study demonstrates that TIGERScore's correlation with human ratings is high and approaches that of GPT-4 evaluators. Human evaluation of the generated explanations showed an accuracy of 70.8%. The researchers believe that TIGERScore shows the potential for building universal explainable metrics to evaluate any NLG task.\n\n2. EvalLM: Interactive System for Iteratively